In [ ]:
import os
from PIL import Image
import numpy as np

In [ ]:
from typing import Coroutine
cover_image_path = './cover.jpg'
data_image_path = './data.jpg'
# cover_image_path = './cover.png'
# data_image_path = './data.png'

if not (os.path.exists(cover_image_path) and os.path.exists(data_image_path)):
    raise ValueError('cover_image_path: "{}" or data_image_path: "{}" is not exist'.format(cover_image_path, data_image_path))

In [ ]:
pixel_bit = 8
bit_n = 3
stegano_bit = {
    1: {
        "cover_mask": 0b11111110,
        "data_mask": 0b00000001,
    },
    2: {
        "cover_mask": 0b11111100,
        "data_mask": 0b00000011,
    },
    3: {
        "cover_mask": 0b11111000,
        "data_mask": 0b00000111,
    },
    4: {
        "cover_mask": 0b11110000,
        "data_mask": 0b00001111,
    },
}

In [ ]:
cover_image = Image.open(cover_image_path)
data_image = Image.open(data_image_path)

In [ ]:
cover_image

In [ ]:
data_image

In [ ]:
cover_image_array = np.array(cover_image)
data_image_array = np.array(data_image)
print('cover image size: ', cover_image_array.shape)
print(' data image size: ', data_image_array.shape)

cover image size:  (1440, 2560, 3)
 data image size:  (1440, 2560, 3)


In [ ]:
if cover_image_array.shape[0] < data_image_array.shape[0] or cover_image_array.shape[1] < data_image_array.shape[1]:
    raise ValueError('data image size is over the cover image size.')

In [ ]:
cover_mask = stegano_bit[bit_n]["cover_mask"]
data_mask = stegano_bit[bit_n]["data_mask"]
stegano_image_array = cover_image_array
for y in range(data_image_array.shape[0]):
    for x in range(data_image_array.shape[1]):
        stegano_image_array[y, x, 0] = (cover_image_array[y, x, 0] & cover_mask) + (data_image_array[y, x, 0] >> (pixel_bit - bit_n))
        stegano_image_array[y, x, 1] = (cover_image_array[y, x, 1] & cover_mask) + (data_image_array[y, x, 1] >> (pixel_bit - bit_n))
        stegano_image_array[y, x, 2] = (cover_image_array[y, x, 2] & cover_mask) + (data_image_array[y, x, 2] >> (pixel_bit - bit_n))

In [ ]:
stegano_image = Image.fromarray(stegano_image_array)
stegano_image

In [ ]:
stegano_image_path = './stegano.png'
stegano_image.save(stegano_image_path)

In [ ]:
stegano_image_array = np.array(Image.open(stegano_image_path))

In [ ]:
extract_image_array = np.zeros_like(stegano_image_array)
if extract_image_array.shape[2] == 4:
    extract_image_array[:, :, 3] = 255
for y in range(stegano_image_array.shape[0]):
    for x in range(stegano_image_array.shape[1]):
        extract_image_array[y, x, 0] = (stegano_image_array[y, x, 0] << (pixel_bit - bit_n)) 
        extract_image_array[y, x, 1] = (stegano_image_array[y, x, 1] << (pixel_bit - bit_n))
        extract_image_array[y, x, 2] = (stegano_image_array[y, x, 2] << (pixel_bit - bit_n))

In [ ]:
extract_image = Image.fromarray(extract_image_array)
extract_image

In [ ]:
extract_image.save('./stegano_decode.png')